In [46]:
import os
import sys
import subprocess
from pathlib import Path

def run_cmd(cmd, shell=False):
    """Helper to run shell commands safely."""
    return subprocess.run(cmd, shell=shell, check=True, capture_output=True)

def download_file(url, output):
    """Attempts to download a file using curl, falling back to wget."""
    try:
        subprocess.run(["curl", "-fsSL", url, "-o", output], check=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        try:
            subprocess.run(["wget", "-q", url, "-O", output], check=True)
        except (subprocess.CalledProcessError, FileNotFoundError):
            raise RuntimeError("Both curl and wget failed. Please install one of them.")

def setup_environment():
    # 1. Detect Environment
    is_colab = 'google.colab' in sys.modules
    is_kaggle = 'KAGGLE_URL_BASE' in os.environ
    is_cloud = is_colab or is_kaggle
    
    # 2. Install uv if not present
    try:
        subprocess.run(["uv", "--version"], capture_output=True, check=True)
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("Installing uv...")
        uv_install_script = "https://astral.sh"
        try:
            # Try installing via curl piped to sh
            subprocess.run(f"curl -LsSf {uv_install_script} | sh", shell=True, check=True)
        except subprocess.CalledProcessError:
            # Fallback to wget piped to sh
            subprocess.run(f"wget -qO- {uv_install_script} | sh", shell=True, check=True)
        
        os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.local/bin")

    # 3. Cloud Logic (Colab/Kaggle)
    if is_cloud:
        platform = "Google Colab" if is_colab else "Kaggle"
        print(f"{platform} detected. Fetching pyproject.toml from GitHub...")
        GITHUB_URL = "https://raw.githubusercontent.com/rooflmaoo-web/eda-video-games-sales/refs/heads/main/pyproject.toml"
        download_file(GITHUB_URL, "pyproject.toml")
        
        print("Syncing cloud environment via uv...")
        subprocess.run(["uv", "pip", "install", "--system", "-r", "pyproject.toml"], check=True)
        
    # 4. Local Logic (VS Code)
    else:
        print("Local environment detected. Running uv sync...")
        subprocess.run(["uv", "sync"], check=True)
        
        project_dir = Path.cwd()
        venv_path = project_dir / ".venv"
        is_running_from_venv = str(venv_path) in sys.executable

        kernel_name = project_dir.name
        display_name = f"uv ({kernel_name})"
        subprocess.run([
            "uv", "run", "python", "-m", "ipykernel", "install", 
            "--user", "--name", kernel_name, "--display-name", display_name
        ], check=True)

        if not is_running_from_venv:
            print("\n" + "!"*60)
            print(f"🚨 ACTION: Switch Kernel to '{display_name}'")
            print("!"*60)
        else:
            print(f"✅ Active Kernel: '{display_name}'")

setup_environment()


Local environment detected. Running uv sync...
✅ Active Kernel: 'uv (eda-video-games-sales)'


In [47]:
def defs(x, s='*'):
    if s != '*':
        print([y for y in dir(x) if y.find(s)>=0])
    else:
        print([y for y in dir(x) if not y.startswith('_')])


<center><h1>EDA of a Video Games Sales Dataset</h1></center>

## Introduction

This notebook is an EDA about a video games sales dataset. The main goals of this EDA are to find out which genres and regions yield the highest revenue, and build a predictive model to assess which genres and regions will yield the highest revenue in five years.

This EDA is divided into four parts:
- Part 1: Data Preparation — preparing the dataset to be used in the analysis.
- Part 2: Structural EDA — understanding the structure of the features in the data.
- Part 3: Feature-Level Analysis — univariate and bivariate analysis, and feature engineering.
- Part 4: Predictions — predicting revenues using modelling.


### Dataset overview
The dataset being used is about sales of video games across different regions. It was taken from [Video Games Sales](https://www.kaggle.com/datasets/lamskdna/video-games-sales), which was posted by Prithu Verma.  
Quoting from the Kaggle page: "This dataset provides a detailed view of video game sales performance, critical reception, and developer/publisher data across multiple gaming platforms. It’s perfect for exploring how factors like genre, publisher reputation, and critic scores influence global sales — and for building predictive models in data science and machine learning."


### Tasks
- Find out which genres and regions yield the highest revenue.
- Build a predictive model to assess which genres and regions will yield the highest revenue in five years.

## Data preparations

In this part the environment is configured, and the dataset is imported, examined, and standardized.

### Configure the environment and import modules.

In [48]:
# Import all modules used by this notebook

import numpy as np
# Interacting with the dataset
import pandas as pd

# Interactive data visualization
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

# Detecting and fixing encoding
from ftfy import fix_and_explain

# Detecting Jupyter theme
from jupyter_dark_detect import is_dark


In [49]:
# Set a global format for floating point numbers
pd.set_option('display.float_format', '{:.2f}'.format)

# Dark/light theme
pio.templates.default = 'plotly_dark' if is_dark() else 'plotly_white'

# Sequential palette for continuous metrics
px.defaults.color_continuous_scale = px.colors.sequential.Cividis

# Discrete palette for categorical metrics
px.defaults.color_discrete_sequence = px.colors.qualitative.Safe


### Import the dataset

In [50]:
# Load the dataset and store it in a DataFrame called df
CSV_URL = 'http://raw.githubusercontent.com/rooflmaoo-web/videos-games-sales-eda/refs/heads/main/VideoGames_Sales.csv'
df = pd.read_csv(CSV_URL)
# Display the first 5 rows
display(df.head())


,title,console,genre,publisher,developer,critic_score,total_sales(mil),na_sales(mil),jp_sales(mil),pal_sales(mil),other_sales(mil),release_date
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.40,$20.32,$6.37,$0.99,$9.85,$3.12,9/17/2013
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.70,$19.39,$6.06,$0.60,$9.71,$3.02,11/18/2014
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.60,$16.15,$8.41,$0.47,$5.49,$1.78,10/28/2002
3,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,$15.86,$9.06,$0.06,$5.33,$1.42,9/17/2013
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.10,$15.09,$6.18,$0.41,$6.05,$2.44,11/6/2015


A short overview of the features in the dataset. Quoting from the Kaggle page:
|Feature|Description|
|---|---|
|title|Name of the video game|
|console|Platform or console on which the game was released (e.g., PS4, X360, PC)|
|genre|Game genre (e.g., Action, Shooter, Sports)|
|publisher|Publishing company responsible for releasing the game|
|developer|Game development studio|
|critic_score|Average critic rating (scale of 0–10)|
|total_sales(mil)|Total worldwide sales in millions of units|
|na_sales(mil)|Sales in North America (millions)|
|jp_sales(mil)|Sales in Japan (millions)|
|pal_sales(mil)|Sales in PAL regions (Europe, Australia, etc.) (millions)|
|other_sales(mil)|Sales in other regions (millions)|
|release_date|Official release date of the game|

### Examine the dataset

In [51]:
# Print the shape of the dataset
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns in the dataset')


There are 64016 rows and 12 columns in the dataset


In [52]:
# Print a short summary of the dataset
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               64016 non-null  object 
 1   console             64016 non-null  object 
 2   genre               64016 non-null  object 
 3   publisher           64016 non-null  object 
 4   developer           63999 non-null  object 
 5   critic_score        6678 non-null   float64
 6    total_sales(mil)   18922 non-null  object 
 7    na_sales(mil)      12637 non-null  object 
 8    jp_sales(mil)      6726 non-null   object 
 9    pal_sales(mil)     12824 non-null  object 
 10   other_sales(mil)   15128 non-null  object 
 11  release_date        56965 non-null  object 
dtypes: float64(1), object(11)
memory usage: 5.9+ MB


In [53]:
# Print the number of duplicated rows
print(f'There are {df.duplicated().sum()} duplicated rows')


There are 21 duplicated rows


In [54]:
# Display a short statistical summary of non-numeric columns
display(df.describe(include=['O']))


,title,console,genre,publisher,developer,total_sales(mil),na_sales(mil),jp_sales(mil),pal_sales(mil),other_sales(mil),release_date
count,64016,64016,64016,64016,63999,18922,12637,6726,12824,15128,56965
unique,39798,81,20,3383,8862,482,320,121,256,133,7922
top,Plants vs. Zombies,PC,Misc,Unknown,Unknown,$0.01,$0.04,$0.01,$-,$-,1/1/1994
freq,17,12617,9304,8842,4435,1366,651,1137,2245,5165,515


In [55]:
# Examine string columns to detect encoding artifacts

# Print strings containing non-ASCII characters
def print_nonascii(df, nrows=5):
    df = df.select_dtypes(['O'])
    for col in df:
        # Drop missing values
        notna = df[col].dropna()
        # Flag strings containing only ASCII character as True
        mask = notna.apply(lambda x: x.isascii())
        # Skip column if all values contain only ASCII characters
        if mask.all():
            continue
        # Flip the mask to filter rows containing only ASCII characters
        noneng = notna[~mask]
        print(f'"{col}" contains non-ASCII characters:\n{noneng.head(nrows)}\n')

print_nonascii(df)


"title" contains non-ASCII characters:
1254    Pok√©mon Mystery Dungeon: Red Rescue Team (US ...
1457                                    Pok√©mon Conquest
1729                      Pok√©mon Card GB2: GRdan Sanjou
1973                                Super Robot Taisen Œ±
2062                                        Pok√©mon Dash
Name: title, dtype: object

"publisher" contains non-ASCII characters:
19500                     Sega¬†AM7
20541    CGE¬†Services¬†Corporation
21027            Capstone¬†Software
21032            Capstone¬†Software
24172          The Pok√©mon Company
Name: publisher, dtype: object

"developer" contains non-ASCII characters:
758                            EA¬†Tiburon
840                             EA¬†Canada
1693             Black¬†Ops¬†Entertainment
2091    Acclaim¬†Studios¬†Salt¬†Lake¬†City
2365                            EA¬†Canada
Name: developer, dtype: object



In [56]:
# Examine the "console" and "genre" columns for spelling or formatting inconsistencies

consoles = sorted(df['console'].unique())
print('Consoles:')
step = 8
for x in range(0, len(consoles), step):
    print(*consoles[x:x+step], sep='\t')

print('\nGenres:')
print(*sorted(df['genre'].unique()), sep='\n')


Consoles:
2600	3DO	3DS	5200	7800	ACPC	AJ	AST
Aco	All	Amig	And	ApII	Arc	BBCM	BRW
C128	C64	CD32	CDi	CV	DC	DS	DSi
DSiW	FDS	FMT	GB	GBA	GBC	GC	GEN
GG	GIZ	Int	Linux	Lynx	MS	MSD	MSX
Mob	N64	NES	NG	NGage	NS	OR	OSX
Ouya	PC	PCE	PCFX	PS	PS2	PS3	PS4
PS5	PSN	PSP	PSV	S32X	SAT	SCD	SNES
Series	TG16	VB	VC	WS	WW	Wii	WiiU
WinP	X360	XB	XBL	XOne	XS	ZXS	iOS
iQue

Genres:
Action
Action-Adventure
Adventure
Board Game
Education
Fighting
MMO
Misc
Music
Party
Platform
Puzzle
Racing
Role-Playing
Sandbox
Shooter
Simulation
Sports
Strategy
Visual Novel


#### Observations
- There are 64016 rows and 12 columns in the dataset.
- Sales column names contain leading whitespace characters, and their data type is object instead of float.
- There are 21 duplicated rows.
- The data type of the `release_date` column is object instead of DateTime.
- Sales and `critic_score` columns contain many missing values.
- Sales columns contain invalid numeric values (e.g., $-).
- `publisher` and `developer` have missingness labeled as *Unknown*.
- `title`, `publisher`, and `developer` contain non-normalized string values.


### Normalize and standardize the dataset

In [57]:
# Strip leading and trailing whitespace characters from column names
df.columns = df.columns.str.strip()
# Re-check column names
print(df.columns)


Index(['title', 'console', 'genre', 'publisher', 'developer', 'critic_score',
       'total_sales(mil)', 'na_sales(mil)', 'jp_sales(mil)', 'pal_sales(mil)',
       'other_sales(mil)', 'release_date'],
      dtype='object')


In [58]:
# Verify duplicated rows can be safely removed
display(df[df.duplicated(keep=False)].head(4))


,title,console,genre,publisher,developer,critic_score,total_sales(mil),na_sales(mil),jp_sales(mil),pal_sales(mil),other_sales(mil),release_date
2438,Zaidan Houjin Nippon Kanji Nouryoku Kentei Kyo...,DS,Misc,IE Institute,IE Institute,NaN,$0.66,NaN,$0.66,NaN,NaN,11/9/2006
2439,Zaidan Houjin Nippon Kanji Nouryoku Kentei Kyo...,DS,Misc,IE Institute,IE Institute,NaN,$0.66,NaN,$0.66,NaN,NaN,11/9/2006
11688,Itoi Shigesato no Bass Tsuri No. 1 Ketteihan!,N64,Sports,Nintendo,HAL Laboratory,NaN,$0.07,NaN,$0.07,NaN,NaN,3/31/2000
11689,Itoi Shigesato no Bass Tsuri No. 1 Ketteihan!,N64,Sports,Nintendo,HAL Laboratory,NaN,$0.07,NaN,$0.07,NaN,NaN,3/31/2000


In [59]:
# Only the index's value changes between duplicates, so drop duplicated rows

df = df.drop_duplicates(ignore_index=True)
# Re-check new shape
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns in the dataset')


There are 63995 rows and 12 columns in the dataset


In [60]:
# Examine invalid numeric values in sales columns

sales_cols = ['total_sales(mil)', 'na_sales(mil)', 'jp_sales(mil)', 'pal_sales(mil)', 'other_sales(mil)']

# Loop over sales columns
for col in sales_cols:
    # Replace the currency sign ($) with an empty string, and coerce invalid values to NaN
    sales = pd.to_numeric(df[col].str.replace('$', ''), errors='coerce')
    # Inspect invalid values
    print(f'{col}: {df.loc[sales.isna(), col].unique()}')


total_sales(mil): [' $-   ' nan]
na_sales(mil): [nan ' $-   ']
jp_sales(mil): [nan ' $-   ']
pal_sales(mil): [' $-   ' nan]
other_sales(mil): [nan ' $-   ']


In [61]:
# The values above aren't valid numeric values.
# Convert sales to float and coerce invalid values to NaN

sales_cols = ['total_sales(mil)', 'na_sales(mil)', 'jp_sales(mil)', 'pal_sales(mil)', 'other_sales(mil)']

for col in sales_cols:
    df[col] = pd.to_numeric(df[col].str.replace('$', ''), errors='coerce')

# Re-check sales dtypes
df[sales_cols].dtypes


total_sales(mil)    float64
na_sales(mil)       float64
jp_sales(mil)       float64
pal_sales(mil)      float64
other_sales(mil)    float64
dtype: object

In [62]:
# Detect the correct encoding to use for non-ASCII string

pokemon = 'Pok√©mon'
print(fix_and_explain(pokemon))


ExplainedText(text='Pokémon', explanation=[('encode', 'macroman'), ('decode', 'utf-8')])


In [63]:
# Fix encoding

cols = ['title', 'publisher', 'developer']
for col in cols:
    # Use x == x to ignore NaN values. 
    df[col] = df[col].apply(lambda x: x.encode('macroman').decode('utf-8') if x == x else x)

print_nonascii(df)


"title" contains non-ASCII characters:
1254    Pokémon Mystery Dungeon: Red Rescue Team (US w...
1457                                     Pokémon Conquest
1729                       Pokémon Card GB2: GRdan Sanjou
1973                                 Super Robot Taisen α
2062                                         Pokémon Dash
Name: title, dtype: object

"publisher" contains non-ASCII characters:
19497                    Sega AM7
20538    CGE Services Corporation
21023           Capstone Software
21028           Capstone Software
24166         The Pokémon Company
Name: publisher, dtype: object

"developer" contains non-ASCII characters:
758                         EA Tiburon
840                          EA Canada
1693           Black Ops Entertainment
2091    Acclaim Studios Salt Lake City
2365                         EA Canada
Name: developer, dtype: object



In [64]:
# Examine the role of "Unknown" in the "publisher" and "developer" columns.

cols = ['publisher', 'developer']

for col in cols:
    s = df[col]
    mask = s.str.contains('unknown', case=False, na=False)
    print(f'{col}: {s[mask].unique()}')


publisher: ['Unknown' 'Unknown Worlds Entertainment']
developer: ['Unknown' 'Unknown Worlds Entertainment']


In [65]:
# "Unknown" may also refer to "Unknown Worlds Entertainment".
# Check if there are observations that need to be fixed.

cols = ['publisher', 'developer']

# Examine observations containing "Unknown Worlds Entertainment"
mask = df[cols].apply(lambda x: x == 'Unknown Worlds Entertainment')
mask = mask.apply(lambda x: x.any(), axis=1)
display(df[mask])


,title,console,genre,publisher,developer,critic_score,total_sales(mil),na_sales(mil),jp_sales(mil),pal_sales(mil),other_sales(mil),release_date
9428,Subnautica,PS4,Adventure,Gearbox Software,Unknown Worlds Entertainment,NaN,0.12,0.07,NaN,0.03,0.02,12/4/2018
11896,Subnautica,XOne,Adventure,Gearbox Software,Unknown Worlds Entertainment,NaN,0.07,0.06,NaN,NaN,0.01,12/4/2018
24649,Subnautica,All,Adventure,Unknown Worlds Entertainment,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,1/23/2018
24650,Subnautica,Series,Adventure,Unknown Worlds Entertainment,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,1/23/2018
24673,Subnautica,PC,Adventure,Unknown Worlds Entertainment,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,1/23/2018
30370,Subnautica: Below Zero,PC,Action-Adventure,Unknown,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30371,Subnautica: Below Zero,PS4,Action-Adventure,Unknown,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30372,Subnautica: Below Zero,PS5,Action-Adventure,Unknown,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30373,Subnautica: Below Zero,XOne,Action-Adventure,Unknown,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30374,Subnautica: Below Zero,XS,Action-Adventure,Unknown,Unknown Worlds Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*Subnautica: Below Zero* and *Natural Selection 2* were originally published by *Unknown Worlds Entertainment*. However, it's not clear whether these observations refer to digital or physical copies, so leave them as *Unknown*.

In [66]:
# Replace "Unknown" values with NaN

df[cols] = df[cols].replace('Unknown', pd.NA)

# Verify
display(df[cols].describe())


,publisher,developer
count,55153,59544
unique,3382,8861
top,Sega,Konami
freq,2205,976


### Load and standardize the dataset

Create a function that loads, standardizes, and normalizes the dataset.

In [67]:
def load_dataset():
    # Read the dataset into a DataFrame, and parse the "release_date" column to DateTime
    df = pd.read_csv(CSV_URL, parse_dates=['release_date'])

    # Strip leading and trailing whitespace characters from column names
    df.columns = df.columns.str.strip()
    
    # Remove duplicated rows
    df = df.drop_duplicates(ignore_index=True)
    
    # Convert sales columns to float
    sales_cols = ['total_sales(mil)', 'na_sales(mil)', 'jp_sales(mil)', 'pal_sales(mil)', 'other_sales(mil)']
    for col in sales_cols:
        df[col] = pd.to_numeric(df[col].str.replace('$', ''), errors='coerce')
    
    # Fix encoding
    cols = ['title', 'publisher', 'developer']
    for col in cols:
        df[col] = df[col].apply(lambda x: x.encode('macroman').decode('utf-8') if x == x else x)
    
    # Replace "Unknown" values with NaN
    cols = ['publisher', 'developer']
    df[cols] = df[cols].replace('Unknown', pd.NA)

    # return the standardized df
    return df


In [68]:
# Load the dataset and verify data types.

df = load_dataset()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63995 entries, 0 to 63994
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             63995 non-null  object        
 1   console           63995 non-null  object        
 2   genre             63995 non-null  object        
 3   publisher         55153 non-null  object        
 4   developer         59544 non-null  object        
 5   critic_score      6678 non-null   float64       
 6   total_sales(mil)  17567 non-null  float64       
 7   na_sales(mil)     12357 non-null  float64       
 8   jp_sales(mil)     6303 non-null   float64       
 9   pal_sales(mil)    10579 non-null  float64       
 10  other_sales(mil)  9963 non-null   float64       
 11  release_date      56944 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(5)
memory usage: 5.9+ MB


The data is now standardized and ready to be explored. Before looking into distributions and relationships, its integrity and validity will be checked.

## Structural EDA

This part is about checking the integrity and validity of the dataset. The main goal is to examine the structure of features by observing the validity of their statistical characteristics, and their availability or missingness.

### Statistical characteristics

In [69]:
# Display a short statistical summary of the data

# Numeric columns
display(df.describe(exclude=['O']))
# Non-numeric columns
display(df.describe(include=['O']))


,critic_score,total_sales(mil),na_sales(mil),jp_sales(mil),pal_sales(mil),other_sales(mil),release_date
count,6678.00,17567.00,12357.00,6303.00,10579.00,9963.00,56944
mean,7.22,0.38,0.27,0.11,0.18,0.07,2006-11-14 14:25:12.222534400
min,1.00,0.01,0.01,0.01,0.01,0.01,1971-12-03 00:00:00
25%,6.40,0.05,0.05,0.02,0.02,0.01,2001-03-29 00:00:00
50%,7.50,0.14,0.12,0.05,0.06,0.02,2008-09-16 00:00:00
75%,8.30,0.38,0.28,0.12,0.17,0.06,2012-12-27 00:00:00
max,10.00,20.32,9.76,2.13,9.85,3.12,2024-12-31 00:00:00
std,1.46,0.83,0.50,0.17,0.43,0.15,NaN


,title,console,genre,publisher,developer
count,63995,63995,63995,55153,59544
unique,39798,81,20,3382,8861
top,Plants vs. Zombies,PC,Misc,Sega,Konami
freq,17,12609,9301,2205,976


#### Observations
- `critic_score`, `sales` - continuous numerical; sales are the target variables; exhibit a high level of missingness.
- `release_date` - ordinal or interval; exhibits a moderate level of missingness.
- `title` - identifier; many observations with duplicated titles; no missing values.
- `console` - nominal; moderate to high cardinality; no missing values.
- `genre` - nominal; *Misc* pending clarification; no missing values.
- `publisher`, `developer` - nominal or identifier; moderate to high level of missingness.

### Structure of missingness

#### Missingness in columns

In [70]:
# Calculate missingness as percentages

missing_pct = df.isna().mean() * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)

print("Columns with missing values (%):")
for col, pct in missing_pct.items():
    print(f"{col:<20} {pct:.0f}%")


Columns with missing values (%):
jp_sales(mil)        90%
critic_score         90%
other_sales(mil)     84%
pal_sales(mil)       83%
na_sales(mil)        81%
total_sales(mil)     73%
publisher            14%
release_date         11%
developer            7%


##### Observations
Out of 12 columns 9 contain missing values, with `sales` and `critic_score` exhibiting a significant degree of missingness. As noted, the sales columns are the target variables, therefore understanding the structure of their missingness may prove to be important for later parts. With *Unknown* values dropped, the `publisher` and `developer` columns now contain a moderate number of missing values, as does the `release_date` column.

#### Sales availability across dataset

In [71]:
# Calculate the availability of sales across all observation

sales_cols = ['total_sales(mil)', 'na_sales(mil)', 'jp_sales(mil)', 'pal_sales(mil)', 'other_sales(mil)']

nsales = df.notna()[sales_cols] \
    .apply(sum, axis=1) \
    .value_counts(normalize=True) * 100

print("Observations per number of sales (%):")
for idx, val in nsales.items():
    print(f"{idx}  {"sales":<10} {val:.2f}%")


Observations per number of sales (%):
0  sales      72.55%
4  sales      9.45%
2  sales      8.89%
3  sales      6.01%
5  sales      3.00%
1  sales      0.10%


##### Observations
The vast majority of observations (72.5%) have no sales values. The rest of the observations are primarily within the range of 2 to 4 sales values. As the sales columns are the target variables, their degree of missingness may prove to be problematic.

#### Sales availability over time

In [72]:
# Calculate percentage of availability in each column, grouped by year

sales_cols = ['total_sales(mil)', 'na_sales(mil)', 'jp_sales(mil)', 'pal_sales(mil)', 'other_sales(mil)']

df_av = (
    df.notna()
    .groupby(df['release_date'].dt.year)
    .mean()
)

fig = px.line(
    df_av,
    y=sales_cols,
    labels={
        "value": "Availability",
        "release_date": "Year",
        "variable": "Sales column"
    },
    title="Sales Availability vs Year"
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Availability",
    yaxis_tickformat=".0%"
)

fig.show()

##### Observations
Sales availability is largely concentrated between approximately mid-1990s to late 2010s. Outside this window, availability remains close to zero, with sparse and low-coverage observations appearing primarily in the late 1970s to mid-1980s. *JP* and *Other* sales availability is lower than the rest, with *JP* having the lowest coverage overall.

#### Sales availability by console

In [73]:
# calculate percentage of availability in each column, grouped by console
df_av = df.notna().groupby(df['console']).mean()
# sort by mean of sales mean across each row, focusing on sales columns
df_av['sales_mean'] = df_av[sales_cols].mean(axis=1)
df_av = df_av.sort_values('sales_mean', ascending=False)[sales_cols]

fig, ax = plt.subplots(figsize=(10,12))

# genereate a heatmap of sales availability across all consoles
img = plt.imshow(
    df_av[sales_cols],
    cmap='RdYlGn', aspect='auto',
    interpolation='nearest',
    vmin=0, vmax=1
)

# add a color bar to the side of the graph, showing the gradient legend
cbar = fig.colorbar(img, ticks=[0, 0.25, 0.5, 0.75,1], ax=ax)
cbar.set_ticklabels(['0%', '25%', '50%', '75%', '100%'])
cbar.set_label('Availability')

# add the sales column labels
ax.set_xticks(range(len(sales_cols)), sales_cols)
# add the console labels
ax.set_yticks(range(len(df_av)), df_av.index, fontsize=7)

# create a secondary x-axis and place it at the top of graph
secax = ax.secondary_xaxis('top', functions=(lambda x: x, lambda x: x))
secax.set_xticks(range(len(sales_cols)))
secax.set_xticklabels(sales_cols)

# remove grid lines
ax.grid(False)

ax.set_title('Sales Availbility by Console')
plt.show()

NameError: name 'plt' is not defined

##### Observations
- *JP* has a low percentage of availability throughout the vast majority of consoles.
- *Other* has slightly more availability than *JP*. Its availability is focused on different consoles than *NA* and *PAL*
- Consoles exhibit patterns consistent with right-censoring (e.g., PS5), while others show low availability likely attributable to source bias or unobserved historical data (e.g., SNES).
- PC has the highest frequency in the `console` column (12609), but sales availability for PC titles is especially low.

The heatmap above shows that out of 81 consoles only ~15 have a significant sales coverage in the dataset. Many popular consoles (e.g, PS5), including Operating Systems and platforms (e.g, Linux and PSN), have almost no sales availability.

#### Critic scores availability over time

In [ ]:
# calculate percentage of availability in critic_score, grouped by year of release_date
df_av = df.notna().groupby(df['release_date'].dt.year).mean() * 100
plt.figure(figsize=(8, 5))
# focus on sales columns
ax = sns.lineplot(df_av['critic_score'])
ax.set(
    title='Critic Scores Availability vs Year',
    ylabel='Availability',
    xlabel='Year'
)
ax.yaxis.set_major_formatter(ticker.PercentFormatter())
plt.show()

##### Observations
As noted in the statistical characteristics section, the coverage of the `critic_score` column is ~10%. Availability is concentrated between approximately mid-1990s to late 2010s, similar to the sales columns.

### Data validity

#### Genres

In [ ]:
print(*sorted(df['genre'].unique()), sep='\n')

##### Observations
- No obvious spelling or formatting inconsistencies were observed in the `genre` field.
- Has low to moderate cardinality, with 20 unique genre values.
- Fully populated across all observations.
- The taxonomy includes both atomic genres (e.g., Action, Adventure) and a compound genre (Action-Adventure).
- The semantic scope of the *Misc* genre is unclear and may require further clarification.

#### Title, publisher, and developer

##### Observations
A small fraction of values in each column contain encoding artifacts or non-normalized characters (e.g., `Pok√©mon Conquest`), likely due to text encoding issues rather than semantic inconsistency. The `publisher` and `developer` columns contain both identical values and closely related entities (e.g., parent companies and subsidiary studios such as `Ubisoft` and `Ubisoft Paris`).

In [ ]:
f = pd.melt(df, value_vars=sales_cols)
g = sns.FacetGrid(f, col='variable', col_wrap=1, sharex=False, sharey=False)
g.map(sns.histplot, 'value')
g.set_titles(col_template='{col_name}')
plt.show()